In [ ]:
import sys
sys.path.append('/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone')
sys.path.append('/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/tools')

from omegaconf import OmegaConf
from src.models.unet_2d_condition import UNet2DConditionModel
from src.models.unet_3d import UNet3DConditionModel

import torch
import os

In [ ]:
cfg = OmegaConf.load("/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/configs/train/stage2.yaml")
infer_config = OmegaConf.load('/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/configs/inference/inference_v2.yaml')

In [ ]:
denoising_unet = UNet3DConditionModel.from_pretrained_2d(
        "/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/pretrained_weights/stable-diffusion-v1-5",
        cfg.mm_path,
        subfolder="unet",
        unet_additional_kwargs=OmegaConf.to_container(
            infer_config.unet_additional_kwargs
        ),
    ).to(device="cuda")

In [ ]:
denoising_unet.load_state_dict(
        torch.load(
            os.path.join(cfg.stage1_ckpt_dir, f"denoising_unet-{cfg.stage1_ckpt_step}.pth"),
            map_location="cpu",
        ),
        strict=False,
    )

In [ ]:
denoising_unet

In [ ]:
from src.utils.lora_handler import LoraHandler
lora_manager = LoraHandler(use_unet_lora=True, unet_replace_modules=["TemporalTransformerBlock"])
unet_lora_params, _ = lora_manager.add_lora_to_model(
        True, denoising_unet, lora_manager.unet_replace_modules, 0,
        '' , r=64)

In [ ]:
len(unet_lora_params)